# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションのためにエージェントを作成するプロセスを示しています。目指すのは、20の質問ゲームをプレイする能力を持つ言語モデル（LLM）を開発することです。このノートブックでは、特に質問者と回答者の役割を担うエージェントを実装しています。

コーディングの初めに、必要なライブラリ（`immutabledict`や`sentencepiece`）のインストール、Gemma（Googleが開発したPyTorchの言語モデルフレームワーク）リポジトリのクローン、必要な構成ファイルの準備を行います。その後、`main.py`というスクリプトでエージェントのクラス構造を作成しています。

主な手法として、以下の要素が挙げられます。

1. **GemmaFormatter**: プロンプトのフォーマットを管理するクラス。ユーザーの質問とモデルの応答を整理する役割を果たします。
2. **GemmaAgent**: 基本的なエージェントクラスで、LLMと対話し、その応答を処理する機能を提供します。このクラスは質問を生成し、応答を解析するためのメソッドを持っています。
3. **GemmaQuestionerAgent**と**GemmaAnswererAgent**: 質問者と回答者の具体的なエージェントクラスで、それぞれの役割に特化した応答処理を行います。質問者はゲームにおける質問を生成し、回答者は与えられた質問に対して「はい」または「いいえ」で応答します。

使用している主なライブラリは以下の通りです：
- **PyTorch**: モデルの構築とトレーニングに利用。
- **Gemma**: 言語モデルのための構成やクラスを含むフレームワークとして使われています。

最終的に、このノートブックを実行することで、タスクを実行するために必要な`submission.tar.gz`ファイルが生成され、Kaggleのコンペティションに直接提出できるようになります。これにより、LLMが20の質問ゲームをどのように戦略的にプレイし、推論を行う能力を持てるかが実証されることを目指しています。

---


# 用語概説 
以下は、Jupyter Notebookに記載されている内容に基づく、初心者がつまずきそうな専門用語の解説です。特に、本ノートブック特有のドメイン知識や実務経験が求められる用語を中心にしています。

1. **エージェント (Agent)**:
   - 環境内で行動や意思決定を行う自律的なプログラムやシステム。ここでは「質問者エージェント」と「回答者エージェント」があり、それぞれ異なる役割を担っている。

2. **プロンプト (Prompt)**:
   - 言語モデルに与える入力文。質問や指示を含むテキストで、モデルの出力を誘導するために使用される。特に、モデルの応答のコンテキストを設定するために重要。

3. **few-shot examples**:
   - 言語モデルが新しいタスクを学習する際に、非常に少量の例（ショット）を用いてそのタスクを遂行する能力。特に、事前訓練されたモデルに追加学習を行わずに新しいタスクに対応する際に使われる。

4. **Causal Language Model (因果言語モデル)**:
   - 文脈に依存した次の単語を生成することに特化したモデル。ここではGemmaForCausalLMがその役割を果たす。文を逐次生成する際に、前の単語を考慮して次の単語を決定することが特徴。

5. **でフォルトのtorch dtype (Default torch dtype)**:
   - PyTorchにおけるデフォルトで使用されるデータ型。異なる精度（例えばfloat32、float64など）を扱う際に重要であり、メモリ使用量やモデルのパフォーマンスに影響を与える。

6. **Kaggle Simulations (Kaggleシミュレーション)**:
   - Kaggleプラットフォーム上でコンペティションを実行するための特別な環境。ここでは、コンペティションのエージェントを実行するためのパスが設定されている。

7. **Interleave (インターリーブ)**:
   - 2つの異なるシーケンスを交互に組み合わせる処理。ここでは、質問と答えを組み合わせるために使われている。

8. **Tokenizer (トークナイザー)**:
   - テキストを処理してモデルに入力できる形式（トークン）に変換する仕組み。言語モデルが理解できる形式にテキストを変換するため不可欠。

9. **Max new tokens (最大新規トークン)**:
   - モデルが生成する応答の長さを制限するためのパラメータ。過剰なトークン生成を防ぎ、モデルの出力を短く保つために使用される。

10. **Top-p sampling (トップpサンプリング)**:
    - 確率分布の上位p%に基づいて次のトークンを選択する手法。これにより、多様な出力が得られる。

これらの用語を理解することで、ノートブックの内容や機械学習・深層学習での実務におけるエージェント設計の重要な概念が明らかになります。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# This notebook illustrates the agent creation process for the LLM 20 Questions competition.
# Running this notebook produces a submission.tar.gz file, which you can submit directly.

%%bash
# Set up the working environment
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir -p /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

%%writefile submission/main.py
# Main script for the agent

import os
import sys
from pathlib import Path
import contextlib
import itertools
from typing import Iterable
import re

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# Setup system path for both notebook and simulation environments
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

# Define constants
WEIGHTS_PATH = os.path.join(
    KAGGLE_AGENT_PATH if os.path.exists(KAGGLE_AGENT_PATH) else "/kaggle/input",
    "gemma/pytorch/7b-it-quant/2"
)

# Prompt Formatting class
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt:
            self.user(self._system_prompt)
        if self._few_shot_examples:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

# Agent class definitions
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """Set the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("Initializing model")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            self.model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{variant}.ckpt')
            self.model.load_weights(ckpt_path)
            self.model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        return self._parse_response(response, obs)

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        sampler_kwargs = sampler_kwargs or {
            'temperature': 0.01,
            'top_p': 0.1,
            'top_k': 1,
        }
        return self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        return match.group().lower() if match else ''

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("Please ask a yes-or-no question.")
        elif obs.turnType == 'guess':
            self.formatter.user("Now guess the keyword. Surround your guess with double asterisks.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            return match.group() if match else "Is it a person?"
        elif obs.turnType == 'guess':
            return self._parse_keyword(response)

class GemmaAnswererAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give a yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# Agent Creation
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."
few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is it a person?", "**no**",
    "Is it a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]

# Global agent variable to avoid loading both agents simultaneously
agent = None

def get_agent(name: str):
    global agent

    if agent is None:
        if name == 'questioner':
            agent = GemmaQuestionerAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
        elif name == 'answerer':
            agent = GemmaAnswererAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
    assert agent is not None, "Agent not initialized."
    return agent

def agent_fn(obs, cfg):
    if obs.turnType in ["ask", "guess"]:
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    return response if response and len(response) > 1 else "yes"

```

</div>
<div class="column-right">

# 日本語訳

```python
# このノートブックは、LLM 20 Questionsコンペティションのエージェント作成プロセスを示しています。
# このノートブックを実行すると、直接提出できるsubmission.tar.gzファイルが生成されます。

%%bash
# 作業環境をセットアップします
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir -p /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

%%writefile submission/main.py
# エージェントのためのメインスクリプト

import os
import sys
from pathlib import Path
import contextlib
import itertools
from typing import Iterable
import re

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# ノートブックとシミュレーション環境のシステムパスをセットアップします
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

# 定数を定義します
WEIGHTS_PATH = os.path.join(
    KAGGLE_AGENT_PATH if os.path.exists(KAGGLE_AGENT_PATH) else "/kaggle/input",
    "gemma/pytorch/7b-it-quant/2"
)

# プロンプトフォーマット用のクラス
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt:
            self.user(self._system_prompt)
        if self._few_shot_examples:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

# エージェントクラスの定義
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたデータ型にデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            self.model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{variant}.ckpt')
            self.model.load_weights(ckpt_path)
            self.model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        return self._parse_response(response, obs)

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        sampler_kwargs = sampler_kwargs or {
            'temperature': 0.01,
            'top_p': 0.1,
            'top_k': 1,
        }
        return self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        return match.group().lower() if match else ''

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はい・いいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("次はキーワードを推測してください。推測を二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            return match.group() if match else "それは人ですか？"
        elif obs.turnType == 'guess':
            return self._parse_keyword(response)

class GemmaAnswererAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"質問はキーワード{obs.keyword}に関するもので、カテゴリは{obs.category}です。はい・いいえで答え、答えを二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# エージェント作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者からのはい・いいえの質問に応答します。キーワードは特定の人物、場所、または物です。"
few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]

# 並行して両方のエージェントを読み込まないためのグローバルエージェント変数
agent = None

def get_agent(name: str):
    global agent

    if agent is None:
        if name == 'questioner':
            agent = GemmaQuestionerAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
        elif name == 'answerer':
            agent = GemmaAnswererAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
    assert agent is not None, "エージェントが初期化されていません。"
    return agent

def agent_fn(obs, cfg):
    if obs.turnType in ["ask", "guess"]:
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    return response if response and len(response) > 1 else "はい"
```

</div>
</details>

In [ ]:
# このノートブックは、LLM 20 Questionsコンペティションのエージェント作成プロセスを示しています。
# このノートブックを実行すると、直接提出できるsubmission.tar.gzファイルが生成されます。

%%bash
# 作業環境をセットアップします
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir -p /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

%%writefile submission/main.py
# エージェントのためのメインスクリプト

import os
import sys
from pathlib import Path
import contextlib
import itertools
from typing import Iterable
import re

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# ノートブックとシミュレーション環境のシステムパスをセットアップします
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

# 定数を定義します
WEIGHTS_PATH = os.path.join(
    KAGGLE_AGENT_PATH if os.path.exists(KAGGLE_AGENT_PATH) else "/kaggle/input",
    "gemma/pytorch/7b-it-quant/2"
)

# プロンプトフォーマット用のクラス
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt:
            self.user(self._system_prompt)
        if self._few_shot_examples:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

# エージェントクラスの定義
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたデータ型にデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            self.model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH, f'gemma-{variant}.ckpt')
            self.model.load_weights(ckpt_path)
            self.model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        return self._parse_response(response, obs)

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        sampler_kwargs = sampler_kwargs or {
            'temperature': 0.01,
            'top_p': 0.1,
            'top_k': 1,
        }
        return self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        return match.group().lower() if match else ''

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class GemmaQuestionerAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はい・いいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("次はキーワードを推測してください。推測を二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            return match.group() if match else "それは人ですか？"
        elif obs.turnType == 'guess':
            return self._parse_keyword(response)

class GemmaAnswererAgent(GemmaAgent):
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"質問はキーワード{obs.keyword}に関するもので、カテゴリは{obs.category}です。はい・いいえで答え、答えを二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# エージェント作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者からのはい・いいえの質問に応答します。キーワードは特定の人物、場所、または物です。"
few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]

# 並行して両方のエージェントを読み込まないためのグローバルエージェント変数
agent = None

def get_agent(name: str):
    global agent

    if agent is None:
        if name == 'questioner':
            agent = GemmaQuestionerAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
        elif name == 'answerer':
            agent = GemmaAnswererAgent(
                device='cuda:0',
                system_prompt=system_prompt,
                few_shot_examples=few_shot_examples,
            )
    assert agent is not None, "エージェントが初期化されていません。"
    return agent

def agent_fn(obs, cfg):
    if obs.turnType in ["ask", "guess"]:
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    return response if response and len(response) > 1 else "はい"